# Load Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Load Dataset

In [2]:
data_dir = './VPN_Traffic/'

batch_size = 32
img_height = 32
img_width = 32

## Training and Validation Sets

In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    validation_split=0.2,
    subset="training",
    seed=69,
    color_mode="grayscale",
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 190473 files belonging to 6 classes.
Using 152379 files for training.


In [4]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    validation_split=0.2,
    subset="validation",
    seed=42,
    color_mode="grayscale",
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 190473 files belonging to 6 classes.
Using 38094 files for validation.


In [5]:
class_names = train_ds.class_names
print(class_names)

['audio', 'chat', 'email', 'files', 'sftp', 'video']


## Batch Shapes

In [6]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 32, 32, 1)
(32,)


# Dataset Performance Optimization

In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Create The Model

In [9]:
num_classes = 6

model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 1)),
    
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0

# Training

In [10]:
import shutil
logdir='./callbacks_cnn'
 
if os.path.exists(logdir):
    shutil.rmtree(logdir) 
os.mkdir(logdir)
        
output_model_file=os.path.join(logdir,"model.h5")
 
callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,save_best_only=True),
    keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3)
]

In [11]:
history=model.fit(train_ds, epochs=10,validation_data=(val_ds), callbacks=callbacks)

Epoch 1/10
4762/4762 [==============================] - 35s 6ms/step - loss: 0.5247 - accuracy: 0.8565 - val_loss: 0.4681 - val_accuracy: 0.8735
Epoch 2/10
4762/4762 [==============================] - 16s 3ms/step - loss: 0.4513 - accuracy: 0.8776 - val_loss: 0.4332 - val_accuracy: 0.8813
Epoch 3/10
4762/4762 [==============================] - 16s 3ms/step - loss: 0.4270 - accuracy: 0.8840 - val_loss: 0.4170 - val_accuracy: 0.8863
Epoch 4/10
4762/4762 [==============================] - 16s 3ms/step - loss: 0.4143 - accuracy: 0.8881 - val_loss: 0.4028 - val_accuracy: 0.8908
Epoch 5/10
4762/4762 [==============================] - 15s 3ms/step - loss: 0.4060 - accuracy: 0.8905 - val_loss: 0.3968 - val_accuracy: 0.8938
Epoch 6/10
4762/4762 [==============================] - 15s 3ms/step - loss: 0.3988 - accuracy: 0.8926 - val_loss: 0.3918 - val_accuracy: 0.8945
Epoch 7/10
4762/4762 [==============================] - 15s 3ms/step - loss: 0.3933 - accuracy: 0.8945 - val_loss: 0.3887 - val_ac